In [ ]:
import pandas as pd
import os
import numpy as np
import rasterio

In [ ]:
def load_GRAVD_gravity_data(file_path):
    """
    Load and prepare airborne gravity data from GRAV-D text file.

    Parameters:
    file_path (str): Path to the gravity data text file.

    Returns:
    pd.DataFrame: Processed DataFrame with identifier, latitude, longitude, altitude, and gravity columns.
    """
    # Extract file name from path and print it
    file_name = os.path.basename(file_path)
    print(f"Loading file: {file_name}")

    # Load the file, reading only the required columns (lat, long, altitude, gravity)
    gravity_data = pd.read_csv(
        file_path,
        delim_whitespace=True,
        usecols=[2, 3, 4, 5],
        names=['Latitude(DD)', 'Longitude(DD)', 'Altitude(m)', 'Gravity(mGal)'],
        header=None
    )
    
    # Add identifier column
    gravity_data['identifier'] = gravity_data.index

    # Check for large data and ask about downsampling
    num_rows = len(gravity_data)
    print(f"Number of points in dataset: {num_rows}")

    if num_rows > 200000:
        response = input("Data has more than 200,000 points. Do you want to downsample? (y/n): ").strip().lower()
        if response == 'y':
            try:
                interval_input = input("Enter downsampling interval (default is 10): ").strip()
                interval = int(interval_input) if interval_input else 10
                gravity_data = gravity_data.iloc[::interval].reset_index(drop=True)
                print(f"Data downsampled using interval {interval}. New number of rows: {len(gravity_data)}")
            except ValueError:
                print("Invalid interval input. Proceeding without downsampling.")


    # Reorder columns
    gravity_data = gravity_data[['identifier', 'Latitude(DD)', 'Longitude(DD)', 'Altitude(m)', 'Gravity(mGal)']]

    return gravity_data


In [ ]:
def write_icgem_input_chunks(gravity_df, output_folder_path, chunk_size=10000):
    """
    Splits the gravity DataFrame into chunks and saves each chunk to a text file.

    Parameters:
    gravity_df (pd.DataFrame): DataFrame containing 'identifier', 'Latitude(DD)', 'Longitude(DD)', and 'Altitude(m)'.
    output_folder_path (str): Path to the folder where chunked text files will be saved.
    chunk_size (int): Number of rows per file. Default is 10,000.

    Returns:
    None
    """
    os.makedirs(output_folder_path, exist_ok=True)
    input_data = gravity_df[['identifier', 'Latitude(DD)', 'Longitude(DD)', 'Altitude(m)']]
    num_chunks = (len(input_data) // chunk_size) + 1

    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = start_idx + chunk_size
        chunk = input_data[start_idx:end_idx]
        output_file = os.path.join(output_folder_path, f"IndexLatLongAlt_Part{i+1}.txt")
        chunk.to_csv(output_file, sep='\t', index=False, header=False)
        print(f"Chunk {i+1} saved to {output_file}")

In [ ]:
def load_icgem_output_chunks(folder_path):
    """
    Loads and concatenates .dat files from the specified folder that match 
    the naming pattern 'GGM05G_Part*.dat'. Assumes files follow ICGEM format 
    with a header ending in 'end_of_head'.

    Parameters:
    folder_path (str): Path to the folder containing the .dat files.

    Returns:
    pd.DataFrame: Combined DataFrame with columns:
        ['identifier', 'longitude', 'latitude', 'h_over_ell', 'geoid', 'gravity_anomaly']
    """
    # Get a sorted list of relevant .dat files
    file_list = sorted([
        f for f in os.listdir(folder_path)
        if f.startswith("GGM05G_Part") and f.endswith('.dat')
    ])

    dataframes = []

    for filename in file_list:
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r') as file:
            # Skip header lines until 'end_of_head'
            for line in file:
                if 'end_of_head' in line:
                    break

            # Read remaining lines into a DataFrame
            df = pd.read_csv(file, delim_whitespace=True, header=None)

            # Assign column names
            df.columns = [
                "identifier", "longitude", "latitude",
                "h_over_ell", "geoid", "gravity_anomaly"
            ]

            dataframes.append(df)

    # Combine all DataFrames
    icgem_df = pd.concat(dataframes, ignore_index=True)
    return icgem_df


In [ ]:
def load_dem():
    """
    Loads a DEM file, replaces nodata values with NaN, generates lat/lon grids,
    and prints extent information.

    Returns:
    dem_array (np.ndarray): Processed DEM values.
    lon_grid (np.ndarray): Longitude grid.
    lat_grid (np.ndarray): Latitude grid.
    """
    print("\n" + "="*70)
    print("🗺️  Loading DEM for the Modeling Region".center(70))
    print("="*70)
    print("\nInstructions:")
    print("- Please provide the SRTM 1 Arc-Second DEM (.tif format) for your modeling region.")
    print("- Ensure the DEM is within the extent of your gravity data.")
    print("- This DEM should represent the topography for the entire block you're modeling.")
    print("- Commonly used extent sizes are 1° x 1° or 2° x 2° in decimal degrees.")
    print("- DEM spatial resolution should be 1 arc-second (~30 meters).")
    print("="*70 + "\n")

    # Ask user for DEM file path
    dem_path = input("Enter the full path to the DEM file: ").strip()

    # Read DEM file
    with rasterio.open(dem_path) as dem:
        dem_array = dem.read(1).astype(float)
        transform = dem.transform
        bounds = dem.bounds
        height, width = dem_array.shape
        nodata_value = dem.nodata

    # Replace nodata with NaN
    dem_array[dem_array == nodata_value] = np.nan

    # Generate longitude and latitude grids
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    lon_grid, lat_grid = rasterio.transform.xy(transform, rows, cols)
    lon_grid = np.array(lon_grid)
    lat_grid = np.array(lat_grid)

    # Calculate region extent and directions
    min_lat, max_lat = bounds.bottom, bounds.top
    min_lon, max_lon = bounds.left, bounds.right

    lat_dir = "North" if max_lat > 0 else "South"
    lon_dir = "East" if max_lon > 0 else "West"

    # Round for printing
    min_lat_int = round(abs(min_lat))
    max_lat_int = round(abs(max_lat))
    min_lon_int = round(abs(min_lon))
    max_lon_int = round(abs(max_lon))

    # Calculate spatial extent
    latitude_extent = round(abs(max_lat - min_lat), 4)
    longitude_extent = round(abs(max_lon - min_lon), 4)

    # For friendly display: if close to whole number, display as "~1°"
    lat_disp = f"~{round(latitude_extent)}°" if abs(latitude_extent - round(latitude_extent)) < 0.01 else f"{latitude_extent}°"
    lon_disp = f"~{round(longitude_extent)}°" if abs(longitude_extent - round(longitude_extent)) < 0.01 else f"{longitude_extent}°"

    print("\n📌  DEM Extent Information:")
    print(f"- Latitude extent: {min_lat_int}° to {max_lat_int}° {lat_dir}")
    print(f"- Longitude extent: {min_lon_int}° to {max_lon_int}° {lon_dir}")
    print(f"\n📏  Your modeling region spans approximately {lat_disp} x {lon_disp}")
    print("="*70 + "\n")


    return dem_array, lon_grid, lat_grid, latitude_extent, longitude_extent


In [ ]:
def load_inputs():
    """
    Interactively loads GRAV-D data, ICGEM outputs, and DEM file, and prepares them for modeling.

    Returns:
    gravity_df (pd.DataFrame): GRAV-D airborne gravity data.
    icgem_df (pd.DataFrame): ICGEM output data.
    dem_array (np.ndarray): DEM values with nodata replaced as NaN.
    lon_grid (np.ndarray): Longitude grid derived from DEM.
    lat_grid (np.ndarray): Latitude grid derived from DEM.
    """
    # Ask user for GRAV-D gravity file
    gravity_file_path = input("Enter the full path to the GRAV-D gravity file: ").strip()
    gravity_df = load_GRAVD_gravity_data(gravity_file_path)

    print("\n" + "="*70)
    print("🔍  Preparing for ICGEM Input Creation".center(70))
    print("="*70)
    print("\nImportant Notes:")
    print("- To reduce gravity data, an approximate orthometric height of the flight is required.")
    print("- Use the GGM05G geoid model to estimate geoid undulation at the observation points.")
    print("- Visit the ICGEM website to download geoid undulation and gravity anomaly values at those points.")
    print("- Gravity anomaly data will be used in later stages (e.g., for remove-restore steps), so download it now.")
    print("- ⚠️ ICGEM only allows up to 10,000 points per file upload.")
    print("="*70 + "\n")

    
    # Ask if ICGEM input chunks are already created
    user_input = input("Have you already created the ICGEM input chunks? (yes/no): ").strip().lower()
    if user_input == "no":
        icgem_input_path = input("Enter the folder path where you want to save the ICGEM input chunks: ").strip()
        write_icgem_input_chunks(gravity_df, icgem_input_path, chunk_size=10000)
    else:
        print("Skipping chunk creation.")


    # Notes before downloading and naming ICGEM output files
    print("\n" + "="*70)
    print("📥  Downloading ICGEM Outputs".center(70))
    print("="*70)
    print("\nInstructions:")
    print("- Now go to the ICGEM website and upload each of the ICGEM input chunks created earlier.")
    print("- For each chunk, download the required data: geoid undulation and gravity anomaly using the GGM05G model.")
    print("- Save each downloaded file with the following naming convention:")
    print("    👉 GGM05G_Part01.txt, GGM05G_Part02.txt, ..., GGM05G_PartXX.txt")
    print("- Make sure the part numbers are zero-padded to two digits (e.g., 01, 02, ..., 10).")
    print("- This naming convention is necessary for automatic reading and merging of the output files.")
    print("="*70 + "\n")


    # Ask for ICGEM output path
    icgem_output_path = input("Enter the folder path containing ICGEM output files: ").strip()
    icgem_df = load_icgem_output_chunks(icgem_output_path)

    # Loading DEM for modelling region and generating latitude and longitude grids
    dem_array, lon_grid, lat_grid, latitude_extent, longitude_extent = load_dem()


    return gravity_df, icgem_df, dem_array, lon_grid, lat_grid, latitude_extent, longitude_extent


In [ ]:
def perform_point_wise_calculations(calculation_df):
    """
    Perform point-wise gravity-related calculations on a given DataFrame.

    This function computes a series of gravity-related quantities for each point
    in the input DataFrame based on ellipsoidal constants of the WGS 84 reference system.
    
    Calculations include:
        - Normal gravity based on Somigliana's formula
        - Approximate orthometric height (H ≈ h - N)
        - Second-order free-air correction
        - Free Air Anomaly (FAA)
        - Long wavelength correction using GGM gravity anomaly
        - Atmospheric correction for gravitational attraction
        - Final surface gravity anomaly corrected for atmospheric effects

    Parameters
    ----------
    calculation_df : pandas.DataFrame
        Input DataFrame must contain the following columns:
        - 'Latitude(DD)'         : Latitude in decimal degrees
        - 'Altitude(m)'          : Ellipsoidal height in meters
        - 'Geoid_GGM(m)'         : Geoid undulation from GGM in meters
        - 'Gravity(mGal)'        : Observed gravity in mGal
        - 'Gravity_Anomaly_GGM(mGal)' : Gravity anomaly from GGM in mGal

    Returns
    -------
    calculation_df : pandas.DataFrame
        Updated DataFrame with the following additional columns:
        - 'g_normal(mGal)'                   : Normal gravity
        - 'H_approx(m)'                      : Approximate orthometric height
        - 'Free_Air_Correction(mGal)'        : 2nd-order free air correction
        - 'FAA(mGal)'                        : Free Air Anomaly
        - 'gravity_anomaly_s_mw(mGal)'       : Short and medium wavelength gravity anomaly
        - 'delta_atm(mGal)'                  : Atmospheric correction
        - 'gravity_anomaly_s_mw_atm(mGal)'   : gravity anomaly after all corrections upto terrain correction
    """
    # Ellipsoidal Constants (WGS 84)
    omega = 7.292115e-5        # Angular velocity of Earth (rad/s)
    a = 6378137.0              # Semi-major axis (meters)
    b = 6356752.3142           # Semi-minor axis (meters)
    GM = 3986004.418e8         # Geocentric gravitational constant (m^3/s^2)
    f = 1 / 298.257223563      # Flattening
    gamma_eq = 978032.53359    # Normal gravity at equator (mGal)
    gamma_pole = 983218.49378  # Normal gravity at pole (mGal)

    # Normal gravity calculation (Somigliana's formula)
    lat_rad = np.radians(calculation_df['Latitude(DD)'])
    cos_lat = np.cos(lat_rad)
    sin_lat = np.sin(lat_rad)

    numerator = (a * gamma_eq * cos_lat**2) + (b * gamma_pole * sin_lat**2)
    denominator = np.sqrt((a**2 * cos_lat**2) + (b**2 * sin_lat**2))
    calculation_df['g_normal(mGal)'] = numerator / denominator

    # Approximate orthometric height, H = h - N
    calculation_df['H_approx(m)'] = calculation_df['Altitude(m)'] - calculation_df['Geoid_GGM(m)']

    # 2nd order free air correction
    m = (omega**2 * a**2 * b) / GM
    c1 = (-2 * calculation_df['g_normal(mGal)']) / a
    term1 = (1 + f + m - 2 * f * sin_lat**2) * calculation_df['H_approx(m)']
    c2 = 3 * (calculation_df['g_normal(mGal)'] / (a**2))
    term2 = calculation_df['H_approx(m)']**2
    fac = (c1 * term1) + (c2 * term2)
    calculation_df['Free_Air_Correction(mGal)'] = -fac

    # Free Air Anomaly (FAA)
    calculation_df['FAA(mGal)'] = (
        calculation_df['Gravity(mGal)'] 
        + calculation_df['Free_Air_Correction(mGal)'] 
        - calculation_df['g_normal(mGal)']
    )

    # Remove long wavelength effects
    calculation_df['gravity_anomaly_s_mw(mGal)'] = (
        calculation_df['FAA(mGal)'] - calculation_df['Gravity_Anomaly_GGM(mGal)']
    )

    # Atmospheric correction
    H = calculation_df['H_approx(m)']
    calculation_df['delta_atm(mGal)'] = (
        0.871 
        - 1.0298e-4 * H 
        + 5.3105e-9 * H**2
        - 2.1642e-13 * H**3
        + 9.5246e-18 * H**4
        - 2.2411e-22 * H**5
    )

    calculation_df['gravity_anomaly_s_mw_atm(mGal)'] = (
        calculation_df['gravity_anomaly_s_mw(mGal)'] - calculation_df['delta_atm(mGal)']
    )

    return calculation_df


In [ ]:
def plot_dataframe_column_map(df, value_col, 
                              lat_col='Latitude(DD)', lon_col='Longitude(DD)', 
                              title=None, cmap='jet', point_size=10, alpha=0.8, 
                              save_path=None, dpi=300, interactive=True):
    """
    Plots any numeric value from a DataFrame over geographic coordinates.

    Parameters:
    - df: pandas DataFrame containing data
    - value_col: str, name of the column to plot
    - lat_col: str, name of the latitude column (default: 'Latitude(DD)')
    - lon_col: str, name of the longitude column (default: 'Longitude(DD)')
    - title: str, title of the plot (optional)
    - cmap: str, matplotlib colormap name (default: 'jet')
    - point_size: int, size of scatter points (default: 10)
    - alpha: float, transparency of the points (default: 0.8)
    - save_path: str or Path, path to save the figure (optional)
    - dpi: int, resolution of saved figure (default: 300)
    - interactive: bool, if True, ask user whether to save plot

    Returns:
    - fig, ax: Matplotlib figure and axis objects
    """
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
    from matplotlib.ticker import MultipleLocator
    import numpy as np

    vmin = df[value_col].min()
    vmax = df[value_col].max()
    ticks = np.linspace(vmin, vmax, 10)

    fig = plt.figure(figsize=(14, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    sc = ax.scatter(
        df[lon_col],
        df[lat_col],
        c=df[value_col],
        cmap=cmap,
        s=point_size,
        alpha=alpha,
        vmin=vmin,
        vmax=vmax,
        transform=ccrs.PlateCarree()
    )

    # Gridlines
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 12}
    gl.ylabel_style = {'size': 12}
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlocator = MultipleLocator(1)
    gl.ylocator = MultipleLocator(1)

    # Title and colorbar
    if title is None:
        title = f"Map of {value_col}"
    plt.title(title, fontsize=14)

    cbar = plt.colorbar(sc, ax=ax, orientation='vertical', label=value_col, shrink=0.7, ticks=ticks)

    # Ask to save
    if interactive and save_path is None:
        save_choice = input("Do you want to save the plot as JPEG? (yes/no): ").strip().lower()
        if save_choice == "yes":
            save_path = input("Enter full path (including filename.jpeg) to save the JPEG: ").strip()
    
    if save_path:
        if not save_path.lower().endswith(".jpeg"):
            save_path += ".jpeg"
        plt.savefig(save_path, dpi=dpi, bbox_inches='tight')
        print(f"✅ Plot saved to: {save_path}")

    plt.show()
    return fig, ax


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.ticker import MultipleLocator

def plot_2d_array_on_map(
    data_array, 
    lat_grid, 
    lon_grid, 
    title, 
    cmap='jet', 
    unit_label='Value', 
    save_fig=False, 
    filename=None, 
    dpi=300, 
    interactive=True
):
    """
    Plot a 2D data array (e.g., DEM) over a latitude/longitude grid using Cartopy.

    Parameters:
        data_array (2D numpy array): The array to plot (e.g., elevation, temperature).
        lat_grid (2D numpy array): Latitude grid corresponding to the data array.
        lon_grid (2D numpy array): Longitude grid corresponding to the data array.
        title (str): Title for the plot.
        cmap (str): Colormap for the plot (default is 'jet').
        unit_label (str): Label for the colorbar (default is 'Value').
        save_fig (bool): If True, the figure may be saved based on interaction or filename.
        filename (str): Name of the file to save the figure (e.g., 'plot.jpeg').
        dpi (int): Resolution of the saved figure (default is 300).
        interactive (bool): If True and save_fig is True, ask whether to save and prompt for path.
    """
    # Determine min, max, and colorbar ticks
    vmin = data_array.min()
    vmax = data_array.max()
    ticks = np.linspace(vmin, vmax, 10)

    # Create figure and axes
    fig = plt.figure(figsize=(14, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Plot the data
    mesh = ax.pcolormesh(
        lon_grid,
        lat_grid,
        data_array,
        shading='auto',
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        transform=ccrs.PlateCarree()
    )

    # Add gridlines
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 12}
    gl.ylabel_style = {'size': 12}
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlocator = MultipleLocator(0.2)
    gl.ylocator = MultipleLocator(0.2)

    # Add colorbar
    cbar = plt.colorbar(mesh, ax=ax, orientation='vertical', label=unit_label, shrink=0.7)
    cbar.set_ticks(ticks)

    # Add title
    plt.title(title, fontsize=14)

    # Handle save interaction
    if save_fig:
        if interactive and filename is None:
            user_input = input("Do you want to save the plot as JPEG? (yes/no): ").strip().lower()
            if user_input == "yes":
                filename = input("Enter full path (including filename) to save the JPEG: ").strip()

        if filename:
            if not filename.lower().endswith(".jpeg"):
                filename += ".jpeg"
            plt.savefig(filename, dpi=dpi, bbox_inches='tight')
            print(f"✅ Figure saved as '{filename}'")

    # Show plot
    plt.show()
    return fig, ax


In [ ]:
import numpy as np
from scipy.fft import fft2, ifft2, fftshift

def compute_terrain_correction_fft(dem_array, dx=30, dy=30, rho=2670, G=6.67430e-11):
    """
    Compute terrain correction using FFT-based method on a given DEM array.

    Parameters:
    -----------
    dem_array : 2D numpy.ndarray
        The digital elevation model (DEM) grid in meters.
    dx : float, optional
        Grid spacing in the x-direction (east-west) in meters. Default is 30.
    dy : float, optional
        Grid spacing in the y-direction (north-south) in meters. Default is 30.
    rho : float, optional
        Density of the crust in kg/m³. Default is 2670.
    G : float, optional
        Gravitational constant in m³/kg/s². Default is 6.67430e-11.

    Returns:
    --------
    TC_mgal : 2D numpy.ndarray
        Terrain correction in mGal.
    """
    ny, nx = dem_array.shape
    h_q = dem_array.astype(np.float64)

    # Generate index grid
    x = np.arange(-nx//2, nx//2, dtype=np.float64) * dx
    y = np.arange(-ny//2, ny//2, dtype=np.float64) * dy
    X, Y = np.meshgrid(x, y)

    # Compute radial distance and kernel
    r = np.sqrt(X**2 + Y**2)
    r[r == 0] = np.inf  # Avoid division by zero
    K = (1 / r**3) * dx * dy
    K = fftshift(K)

    # Perform FFT-based convolution
    F_HQ2 = fft2(h_q**2)
    F_HQ = fft2(h_q)
    F_K = fft2(K)

    I1 = ifft2(F_HQ2 * F_K).real
    I2 = ifft2(fft2(np.ones_like(h_q)) * F_K).real
    I3 = ifft2(F_HQ * F_K).real

    # Terrain correction in mGal
    TC = (G * rho / 2) * (I1 + h_q**2 * I2 - 2 * h_q * I3)
    TC_mgal = TC * 1e5

    return TC_mgal


In [ ]:
import numpy as np
from scipy.signal import fftconvolve

def compute_terrain_correction_tiled(dem_array, tile_size=1024, pad=100, dx=30, dy=30, rho=2670, G=6.67430e-11):
    """
    Compute terrain correction in tiles using FFT-based convolution to handle large DEM arrays.

    Parameters:
    -----------
    dem_array : 2D numpy.ndarray
        Digital elevation model (DEM) grid in meters.
    tile_size : int, optional
        Size of each square tile (e.g., 1024x1024). Default is 1024.
    pad : int, optional
        Padding size around each tile to minimize edge effects. Default is 100.
    dx, dy : float
        Grid spacing in x and y directions in meters. Defaults are 30 m.
    rho : float, optional
        Density of the crust in kg/m³. Default is 2670.
    G : float, optional
        Gravitational constant in m³/kg/s². Default is 6.67430e-11.

    Returns:
    --------
    full_tc_mgal : 2D numpy.ndarray
        Terrain correction in mGal, same shape as input DEM.
    """
    ny, nx = dem_array.shape
    full_tc_mgal = np.zeros_like(dem_array, dtype=np.float64)

    def compute_tile(h_q):
        ty, tx = h_q.shape
        x = np.arange(-tx//2, tx//2) * dx
        y = np.arange(-ty//2, ty//2) * dy
        X, Y = np.meshgrid(x, y)
        r = np.sqrt(X**2 + Y**2)
        r[r == 0] = np.inf  # Avoid division by zero
        K = (1 / r**3) * dx * dy

        I1 = fftconvolve(h_q**2, K, mode='same')
        I2 = fftconvolve(np.ones_like(h_q), K, mode='same')
        I3 = fftconvolve(h_q, K, mode='same')

        TC = (G * rho / 2) * (I1 + h_q**2 * I2 - 2 * h_q * I3)
        return TC * 1e5  # Convert to mGal

    for y_start in range(0, ny, tile_size):
        for x_start in range(0, nx, tile_size):
            y_end = min(y_start + tile_size, ny)
            x_end = min(x_start + tile_size, nx)

            # Determine padded bounds
            yp_start = max(y_start - pad, 0)
            yp_end = min(y_end + pad, ny)
            xp_start = max(x_start - pad, 0)
            xp_end = min(x_end + pad, nx)

            # Extract padded tile
            padded_tile = dem_array[yp_start:yp_end, xp_start:xp_end]

            # Compute terrain correction on padded tile
            tc_padded = compute_tile(padded_tile)

            # Extract central (unpadded) region
            y0 = y_start - yp_start
            y1 = y0 + (y_end - y_start)
            x0 = x_start - xp_start
            x1 = x0 + (x_end - x_start)

            full_tc_mgal[y_start:y_end, x_start:x_end] = tc_padded[y0:y1, x0:x1]

    return full_tc_mgal

# TC_mgal = compute_terrain_correction_tiled(dem_array, tile_size=1024, pad=100)

In [ ]:
import numpy as np
from scipy.fft import fft2, ifft2, fftshift

def compute_spherical_stokes_convolution(
    faye_anomaly_grid,
    lat_grid,
    lon_grid,
    R=6371000,
    arcsec_resolution=1
):
    """
    Compute the geoid undulation contribution using spherical Stokes convolution
    via FFT based on a grid of Faye gravity anomalies.

    Parameters:
    -----------
    faye_anomaly_grid : 2D numpy.ndarray
        Grid of Faye gravity anomalies (in mGal).
    lat_grid : 2D numpy.ndarray
        Latitude grid in decimal degrees (same shape as faye_anomaly_grid).
    lon_grid : 2D numpy.ndarray
        Longitude grid in decimal degrees (same shape as faye_anomaly_grid).
    R : float, optional
        Mean Earth radius in meters. Default is 6371000.
    arcsec_resolution : float, optional
        Grid resolution in arc-seconds. Default is 1 (i.e., 1 arc-second resolution).

    Returns:
    --------
    T_r_m2_per_s2 : 2D numpy.ndarray
        Result of the Stokes convolution (in m²/s²).
    """
    ny, nx = faye_anomaly_grid.shape

    # Convert to radians
    lat_rad = np.deg2rad(lat_grid).astype(np.float64)
    lon_rad = np.deg2rad(lon_grid).astype(np.float64)

    # Get center point
    i_center = ny // 2
    j_center = nx // 2
    lat_p = lat_rad[i_center, j_center]
    lon_p = lon_rad[i_center, j_center]

    # Compute sin²(ψ/2)
    sin_psi_by_2_squared = (
        np.sin((lat_p - lat_rad) / 2)**2 +
        np.cos(lat_p) * np.cos(lat_rad) * np.sin((lon_p - lon_rad) / 2)**2
    )
    sin_psi_by_2 = np.sqrt(sin_psi_by_2_squared)
    sin_psi_by_2[i_center, j_center] = np.nan  # exclude center point

    # Clamp and sanitize
    sin_psi_by_2 = np.clip(sin_psi_by_2, 0, 1)
    sin_psi_by_2 = np.nan_to_num(sin_psi_by_2, nan=1e-10)
    psi = 2 * np.arcsin(sin_psi_by_2)

    # Compute Stokes kernel S(ψ)
    with np.errstate(divide='ignore', invalid='ignore'):
        S_psi = (
            (1 / sin_psi_by_2) -
            6 * sin_psi_by_2 +
            1 -
            5 * np.cos(psi) -
            3 * np.cos(psi) * np.log(np.maximum(sin_psi_by_2 + sin_psi_by_2_squared, 1e-10))
        )
    S_psi[i_center, j_center] = 0  # Set singular point to zero

    # Compute area element correction (1 arcsec resolution)
    dx = dy = (arcsec_resolution / 3600) * (np.pi / 180)  # arc-sec to radians
    cos_phi = np.cos(lat_rad)
    S_psi *= cos_phi * dx * dy

    # Apply FFT-based convolution
    F_S_psi = fft2(fftshift(S_psi))
    F_faye = fft2(faye_anomaly_grid)
    T_r = (R / (4 * np.pi)) * ifft2(F_faye * F_S_psi).real

    # Convert result to m²/s²
    T_r_m2_per_s2 = T_r * 1e-5

    return T_r_m2_per_s2


In [ ]:
import numpy as np

def compute_normal_gravity_grid(lat_grid):
    """
    Compute the normal gravity on the WGS84 ellipsoid for each point in a latitude grid.

    Parameters:
    -----------
    lat_grid : 2D numpy.ndarray
        Latitude grid in decimal degrees.

    Returns:
    --------
    g_normal_grid : 2D numpy.ndarray
        Grid of normal gravity values (in m/s²).
    """

    # Ellipsoidal constants (WGS 84)
    a = 6378137.0               # Semi-major axis (meters)
    b = 6356752.3142            # Semi-minor axis (meters)
    gamma_eq = 9.7803253359     # Normal gravity at equator (m/s²)
    gamma_pole = 9.8321849378   # Normal gravity at pole (m/s²)

    # Convert latitude to radians
    lat_rad = np.radians(lat_grid)

    # Compute normal gravity
    g_normal_grid = (
        (a * gamma_eq * np.cos(lat_rad)**2) +
        (b * gamma_pole * np.sin(lat_rad)**2)
    ) / (
        np.sqrt((a**2 * np.cos(lat_rad)**2) +
                (b**2 * np.sin(lat_rad)**2))
    )

    return g_normal_grid


In [ ]:
import numpy as np
from scipy.fft import fft2, ifft2, fftshift

def compute_indirect_effect_fft(dem_array, g_normal_grid, dx=30, dy=30, rho=2670, G=6.67430e-11):
    """
    Compute the indirect effect for geoid modeling using FFT-based convolution.

    Parameters:
    -----------
    dem_array : 2D numpy.ndarray
        Digital Elevation Model (DEM) in meters.

    g_normal_grid : 2D numpy.ndarray
        Grid of normal gravity values (in m/s²) for normalization.

    dx : float, optional
        Grid spacing in x-direction (meters). Default is 30.

    dy : float, optional
        Grid spacing in y-direction (meters). Default is 30.

    rho : float, optional
        Density of the crust in kg/m³. Default is 2670.

    G : float, optional
        Gravitational constant in m³ kg⁻¹ s⁻². Default is 6.67430e-11.

    Returns:
    --------
    N_indirect : 2D numpy.ndarray
        Indirect effect in meters.
    """
    # Get dimensions and convert to float64
    ny, nx = dem_array.shape
    h_q = dem_array.astype(np.float64)

    # Generate distance grid
    x = np.arange(-nx // 2, nx // 2, dtype=np.float64) * dx
    y = np.arange(-ny // 2, ny // 2, dtype=np.float64) * dy
    X, Y = np.meshgrid(x, y)

    # Compute radial distance r
    r = np.sqrt(X**2 + Y**2)
    r[r == 0] = np.inf  # Avoid division by zero

    # Compute kernel (1/r^3) and account for area
    K = (1 / r**3) * dx * dy
    K = fftshift(K)

    # Compute FFTs
    F_HQ3 = fft2(h_q**3)
    F_1 = fft2(np.ones_like(h_q))
    F_K = fft2(K)

    # FFT-based convolution terms
    Term1 = ifft2(F_HQ3 * F_K).real
    Term2 = ifft2(F_1 * F_K).real

    # Compute indirect effect
    N_indirect = (-G * rho) * (np.pi * h_q**2 + (1 / 6) * (Term1 - h_q**3 * Term2))

    # Normalize by normal gravity
    N_indirect /= g_normal_grid

    return N_indirect


In [ ]:
import numpy as np
from scipy.signal import fftconvolve

def compute_indirect_effect_tiled(dem_array, g_normal_grid, tile_size=1024, pad=100,
                                   dx=30, dy=30, rho=2670, G=6.67430e-11):
    """
    Compute the indirect effect for geoid modeling using a tiled FFT-based convolution approach.

    Parameters:
    -----------
    dem_array : 2D numpy.ndarray
        Digital Elevation Model (DEM) in meters.
    g_normal_grid : 2D numpy.ndarray
        Normal gravity grid (in m/s²) for normalization.
    tile_size : int
        Size of processing tiles (e.g., 1024).
    pad : int
        Padding added to each tile to minimize edge effects.
    dx, dy : float
        Grid spacing in meters.
    rho : float
        Density in kg/m³. Default 2670.
    G : float
        Gravitational constant in m³/kg/s². Default 6.67430e-11.

    Returns:
    --------
    full_indirect : 2D numpy.ndarray
        Indirect effect in meters.
    """
    ny, nx = dem_array.shape
    full_indirect = np.zeros_like(dem_array, dtype=np.float64)

    def compute_tile(h_q_tile, g_tile):
        ty, tx = h_q_tile.shape
        x = np.arange(-tx // 2, tx // 2) * dx
        y = np.arange(-ty // 2, ty // 2) * dy
        X, Y = np.meshgrid(x, y)

        r = np.sqrt(X**2 + Y**2)
        r[r == 0] = np.inf
        K = (1 / r**3) * dx * dy

        Term1 = fftconvolve(h_q_tile**3, K, mode='same')
        Term2 = fftconvolve(np.ones_like(h_q_tile), K, mode='same')

        N_indirect = (-G * rho) * (np.pi * h_q_tile**2 + (1 / 6) * (Term1 - h_q_tile**3 * Term2))
        N_indirect /= g_tile
        return N_indirect

    for y_start in range(0, ny, tile_size):
        for x_start in range(0, nx, tile_size):
            y_end = min(y_start + tile_size, ny)
            x_end = min(x_start + tile_size, nx)

            # Padded region
            yp_start = max(y_start - pad, 0)
            yp_end = min(y_end + pad, ny)
            xp_start = max(x_start - pad, 0)
            xp_end = min(x_end + pad, nx)

            h_tile_padded = dem_array[yp_start:yp_end, xp_start:xp_end]
            g_tile_padded = g_normal_grid[yp_start:yp_end, xp_start:xp_end]

            n_tile = compute_tile(h_tile_padded, g_tile_padded)

            # Extract central tile from padded output
            y0 = y_start - yp_start
            y1 = y0 + (y_end - y_start)
            x0 = x_start - xp_start
            x1 = x0 + (x_end - x_start)

            full_indirect[y_start:y_end, x_start:x_end] = n_tile[y0:y1, x0:x1]

    return full_indirect

# N_indirect = compute_indirect_effect_tiled(dem_array, g_normal_grid, tile_size=1024, pad=100)

In [ ]:
import pandas as pd

def load_gsvs17_colorado(filepath=None):
    """
    Load GSVS17 (Colorado, 2017) Geoid Slope Validation Survey data.

    This survey evaluates geoid accuracy in a rugged, high elevation, gravimetrically 
    and topographically complex region.

    Parameters:
    -----------
    filepath : str, optional
        Path to the GSVS17 Excel file. If not provided, user will be prompted to input it.

    Returns:
    --------
    pd.DataFrame
        DataFrame containing Latitude, Longitude, Ellipsoid Height, Orthometric Height, and Geometric Geoid.
    """
    if filepath is None:
        filepath = input("Enter file path for GSVS17 (Colorado) Excel file: ")

    df = pd.read_excel(filepath, usecols="J:M")
    df.columns = ['Latitude', 'Longitude', 'Ellipsoid_Height', 'Orthometric_Height']
    df['Longitude'] = df['Longitude'].apply(lambda x: x - 360 if x > 180 else x)
    df['Geometric_Geoid'] = df['Ellipsoid_Height'] - df['Orthometric_Height']
    return df


In [ ]:
def load_gsvs14_iowa(filepath=None):
    """
    Load GSVS14 (Iowa, 2014) Geoid Slope Validation Survey data.

    This survey evaluates geoid accuracy in a flat, high elevation, and gravimetrically complex region.

    Parameters:
    -----------
    filepath : str, optional
        Path to the GSVS14 Excel file. If not provided, user will be prompted to input it.

    Returns:
    --------
    pd.DataFrame
        DataFrame containing Latitude, Longitude, Ellipsoid Height, Orthometric Height, and Geometric Geoid.
    """
    if filepath is None:
        filepath = input("Enter file path for GSVS14 (Iowa) Excel file: ")

    df = pd.read_excel(filepath, usecols="D,H,L,M")
    df.columns = ['Latitude', 'Longitude', 'Ellipsoid_Height', 'Orthometric_Height']
    df['Longitude'] = df['Longitude'].apply(lambda lon: lon - 360 if lon > 180 else lon)
    df['Geometric_Geoid'] = df['Ellipsoid_Height'] - df['Orthometric_Height']
    df.dropna(how='all', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df


In [ ]:
def load_gsvs11_texas(filepath=None):
    """
    Load GSVS11 (Texas, 2011) Geoid Slope Validation Survey data.

    This survey evaluates geoid accuracy in a flat and gravimetrically uncomplicated region.

    Parameters:
    -----------
    filepath : str, optional
        Path to the GSVS11 Excel file. If not provided, user will be prompted to input it.

    Returns:
    --------
    pd.DataFrame
        DataFrame containing Latitude, Longitude, Ellipsoid Height, Orthometric Height, and Geometric Geoid.
    """
    if filepath is None:
        filepath = input("Enter file path for GSVS11 (Texas) Excel file: ")

    df = pd.read_excel(filepath, usecols="I,M,Q,R")
    df.columns = ['Latitude', 'Longitude', 'Ellipsoid_Height', 'Orthometric_Height']
    df['Longitude'] = -df['Longitude'].abs()
    df['Geometric_Geoid'] = df['Ellipsoid_Height'] - df['Orthometric_Height']
    return df
